In [1]:
import numpy as np
import pandas as pd
import seaborn as sns #for plotting
import csv as csv

#for gene names converting
import mygene as mg

#for finding duplicates
import collections

#for data fetching
import xenaPython as xena

# for survival analysis
import lifelines
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter

/home/stephane/anaconda3/lib/python3.7/site-packages/xenaPython/__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


# Fetch the data on TCGA (using XenaPython)

In [2]:
help(mg)

Help on package mygene:

NAME
    mygene - Python Client for MyGene.Info services

PACKAGE CONTENTS


CLASSES
    biothings_client.MyGeneInfo(biothings_client.base.BiothingClient, biothings_client.mixins.gene.MyGeneClientMixin)
        MyGeneInfo
    
    class MyGeneInfo(biothings_client.MyGeneInfo)
     |  MyGeneInfo(url=None)
     |  
     |  This is the client for a biothing web service.
     |  
     |  Method resolution order:
     |      MyGeneInfo
     |      biothings_client.MyGeneInfo
     |      biothings_client.base.BiothingClient
     |      biothings_client.mixins.gene.MyGeneClientMixin
     |      builtins.object
     |  
     |  Methods inherited from biothings_client.MyGeneInfo:
     |  
     |  clear_cache(self)
     |      Clear the globally installed cache.
     |  
     |  get_fields(self, search_term=None, verbose=True)
     |      Return all available fields can be return from MyGene.info services.
     |      
     |      This is a wrapper for http://mygene.info

In [3]:
def get_codes(host, dataset, fields, data):
    "get codes for enumerations"
    codes = xena.field_codes(host, dataset, fields)
    codes_idx = dict([(x['name'], x['code'].split('\t')) for x in codes if x['code'] is not None])
    for i in range(len(fields)):
        if fields[i] in codes_idx:
            data[i] = [None if v == 'NaN' else codes_idx[fields[i]][int(v)] for v in data[i]]
    return data

def get_fields(host, dataset, samples, fields):
    "get field values"
    data = xena.dataset_fetch(host, dataset, samples, fields)
    return data

def get_fields_and_codes(host, dataset, samples, fields):
    "get fields and resolve codes"
    return get_codes( host, dataset, fields, get_fields( host, dataset, samples, fields))

### Gene expression data

In [4]:
dico_genes = mg.get_client("gene")
dico_genes.getgenes


<bound method _getannotations of <biothings_client.MyGeneInfo object at 0x7faf1e45a940>>

In [5]:
expression = np.array([])
test = np.array([1,2])
for i in range(4):
    expression = np.concatenate([expression,test])
expression = expression.tolist()
print(type(expression))
print(expression)

<class 'list'>
[1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0]


In [6]:
hub = "https://gdc.xenahubs.net"
dataset = "TCGA-BRCA/Xena_Matrices/TCGA-BRCA.htseq_fpkm-uq.tsv" #database url was found on the hub website
samples = xena.dataset_samples (hub, dataset, None)
##wave genes
#genes =["NCKAP1","CYFIP2", "NCKAP1L", "WASF2", "ABI3", "WASF3", "ABI1", "ABI2", "CYFIP1", "WASF1", "BRK1"] #enter the genes of interest
##Arp2-3 genes
#genes = ["ACTR2", "ACTR3", "ARPC1A", "ARPC2", "ARPC3", "ARPC4", "ARPC5", "ACTR3B", "ARPC1B", "ARPC5L"]
##Arpin
#genes = ["Arpin"]
## All genes available
genes = xena.dataset_field(hub, dataset)[:-1]
fields = ["sampleID"]

for i in range(len(genes)):
    genes[i] = genes[i].split(".")[0] #What is the meaning of the number after the point ?

#Convert gene names into their symbol
genes_1 = []
dico_genes = mg.get_client("gene").getgenes(genes,'symbol') #Enlever les chiffres après le point dans le ENSG : create duplicates
genes_1 = [d.get('symbol') for d in dico_genes]
genes_1 = [x for x in genes_1 if x is not None]

#Eliminate the duplicates in the gene names
genes_set = set()
genes_diff = [x for x in genes_1 if x not in genes_set and not genes_set.add(x)]
print('len(genes_diff )',len(genes_diff))
print('len(genes_1) ',len(genes_1))

######
expressions_0 = np.array([])

for i in range(1):
    if i % 20 == 0:
        print(i)
    expressions_0 = np.append(expressions_0,np.array(xena.dataset_gene_probe_avg(hub, dataset, samples, genes_diff[min(i*100, len(genes_diff)) : min((i+1)*100, len(genes_diff))])))
    #expressions_0.extend(xena.dataset_gene_probe_avg(hub, dataset, samples, genes_diff[min(i*100, len(genes_diff)) : min((i+1)*100, len(genes_diff))]))
#expressions_0 = xena.dataset_gene_probe_avg(hub, dataset, samples, genes)

expressions_1 = expressions_0.tolist()

with open('expressions_1_partial', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(expressions_1)    
    
values = get_fields_and_codes(hub, dataset, samples, fields) # list of lists

genes_dict_for_df = dict(zip(fields, values)) #dict where we add the gene expression and the sample ids

with open('dict_partial.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in genes_dict_for_df.items():
       writer.writerow([key, value])

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [7]:
for dico in expressions_1:
    if(dico["gene"] != None):
        label = "expression" + dico["gene"]
        scores = dico["scores"][0] #Problem with the filfulling of the scores of expression_0 : it needs probably the usual names of the genes (like Arpin) instead of the scientific names (like ENSG0000000.03) 
        if(len(scores) > 0):
            genes_dict_for_df[label] = scores

### Survival data

In [8]:
dataset = "TCGA-BRCA/Xena_Matrices/TCGA-BRCA.survival.tsv"
fields = ['_TIME_TO_EVENT', "_EVENT", "sampleID"] #event is 1 if the patient has died or 0 if he was censored (lost...)
#TODO
# As in pancan, there are normal samples in tcga which should probably be removed. _sample_type will
# identify normals. _study will identify tcga vs. gtex vs. target.
values = get_fields_and_codes(hub, dataset, samples, fields) # list of lists
survival_dict_for_df = dict(zip(fields, values)) # index by phenotype

In [9]:
survival_dict_for_df["survival"] = survival_dict_for_df.pop("_TIME_TO_EVENT") 
survival_dict_for_df["event"] = survival_dict_for_df.pop("_EVENT")

### Phenotype data

In [10]:
dataset = "TCGA-BRCA/Xena_Matrices/TCGA-BRCA.GDC_phenotype.tsv"
fields = ['sample_type.samples', "sampleID"] 
values = get_fields_and_codes(hub, dataset, samples, fields) # list of lists
phenotype_dict_for_df = dict(zip(fields, values)) # index by phenotype

In [11]:
phenotype_dict_for_df["sample_type"] = phenotype_dict_for_df.pop('sample_type.samples')

Note : I don't know if it's useful to remove the patients whose tumor is not a "Primary Tumor", like advised by Xena. The goal is to remove duplicates but I can't find duplicates

### Merge data

In [12]:
df_genes = pd.DataFrame(data=genes_dict_for_df)
df_survival = pd.DataFrame(data=survival_dict_for_df)
#df_phenotype = pd.DataFrame(data=phenotype_dict_for_df)

df = pd.merge(df_genes, df_survival) # merge the survival and genes data according to sampleID
#df = pd.merge(df, df_phenotype) #merge also the phenotype data 
df = df.convert_objects(convert_numeric=True)
df = df[~pd.isna(df["survival"])][~pd.isna(df["event"])]
#df = df[df["sample_type"] == "Primary Tumor"] #keep only primary tumor to remove duplicate WARNING : check if name doesn't change if you use another dataset
#df = df.drop(['sampleID','sample_type'], axis=1) #remove identifier for analysis

df = df.drop('sampleID', axis=1) #remove identifier for analysis

In [13]:
df.head()

,expressionTSPAN6,expressionTNMD,expressionDPM1,expressionSCYL3,expressionC1orf112,expressionFGR,expressionCFH,expressionFUCA2,expressionGCLC,expressionNFYA,...,expressionMSL3,expressionCREBBP,expressionMPO,expressionPON1,expressionGCFC2,expressionWDR54,expressionCROT,expressionABCB4,survival,event
0,15.64,8.795,19.08,15.82,14.50,14.65,15.29,17.68,16.27,18.46,...,15.86,17.97,6.675,6.988,16.19,15.96,15.81,11.72,4047,0
1,15.67,9.455,19.70,17.87,15.66,15.95,16.90,18.64,16.93,17.62,...,17.08,17.25,10.320,13.870,16.63,17.67,16.03,12.85,4005,0
2,18.12,14.960,20.00,16.03,15.49,16.69,16.85,18.37,16.48,17.65,...,16.38,16.87,0.000,9.447,16.42,17.19,17.82,12.13,1474,0
3,18.12,10.070,18.93,16.62,15.23,15.55,17.12,18.31,16.40,18.26,...,16.40,17.21,9.266,8.263,16.45,16.39,18.03,11.74,1448,0
4,17.97,12.040,19.04,16.30,14.94,15.50,17.43,18.57,16.49,17.68,...,16.59,17.60,10.170,8.322,16.16,16.21,16.95,12.94,348,0


   ## Export the data so we can use them with R

In [14]:
df.to_csv("data_base_partial")

# Analyse the data (using Lifelines)

In [15]:
#kmf = KaplanMeierFitter()
#kmf.fit(df["survival"], df["event"]) #TODO : tell lifelines if the patient is censored or dead 
#kmf.plot()

In [16]:
#cph = CoxPHFitter()
#cph.fit(df, duration_col='survival', event_col='event')


#cph.print_summary() 

In [17]:
#cph.plot()

In [18]:
#cph.plot_covariate_groups('expressionNCKAP1', [0, 5, 10, 15])

## The concordance and log-likelihood indicate that the model do not fit very well : let's check the assumptions for cox regression

In [19]:
#cph.check_assumptions(df)

In [20]:
# doesn't work so go to R

## TODO : do we have to remove tumors that are not Primary tumors from the list ? make the same transformation than Marc Lavielle, check assumption for Cox Regression, use other form of gene expression (RNAseq) to compare, use other datasets

Liens : 

objectifs en cours : https://docs.google.com/document/d/1A5wxgQ_ZPry4aRH1BsmdbmPJfJoIsy5sa-4P5RmWaTs/edit


dossier partagé : https://drive.google.com/drive/u/0/folders/1HotcSiF2BzDuzXMhKNX1p-7lI1U_3UQ9

rapport : https://www.overleaf.com/project/5c34f6b7f99dfe3f36bf3a55


### Generate data for report (formatting and renaming)

In [21]:
#df2 = pd.read_csv("data_for_report_arp23")

In [22]:
#df2

In [23]:
#df2 = df2[["Unnamed: 0", "coef", "Pr(>|z|)"]]

In [24]:
#df2 = df2.rename(columns={"Unnamed: 0": "gene", "coef": "coefficient", "Pr(>|z|)" : "p-value"})

In [25]:
#df2["gene"] = df2["gene"].apply(lambda x: x[10:])

In [26]:
#df2